## Pose2Sim for triangulation

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, FFMpegWriter

import shutil
import glob
import itertools
import yaml
import io
import contextlib

# Test torch installation
import torch; import onnxruntime as ort
print(torch.cuda.is_available(), ort.get_available_providers())
# Should print "True ['CUDAExecutionProvider', ...]"

from Pose2Sim import Pose2Sim

In [ ]:
#copy Pose2Sim calibration file writing to go from matrices to Calib.toml
def toml_write(calib_path, C, S, D, K, R, T):
    '''
    Writes calibration parameters to a .toml file

    INPUTS:
    - calib_path: path to the output calibration file: string
    - C: camera name: list of strings
    - S: image size: list of list of floats
    - D: distorsion: list of arrays of floats
    - K: intrinsic parameters: list of 3x3 arrays of floats
    - R: extrinsic rotation: list of arrays of floats (Rodrigues)
    - T: extrinsic translation: list of arrays of floats

    OUTPUTS:
    - a .toml file cameras calibrations
    '''

    with open(os.path.join(calib_path), 'w+') as cal_f:
        for c in range(len(C)):
            cam=f'[{C[c]}]\n'
            name = f'name = "{C[c]}"\n'
            size = f'size = [ {S[c][0]}, {S[c][1]}]\n' 
            mat = f'matrix = [ [ {K[c][0,0]}, 0.0, {K[c][0,2]}], [ 0.0, {K[c][1,1]}, {K[c][1,2]}], [ 0.0, 0.0, 1.0]]\n'
            dist = f'distortions = [ {D[c][0]}, {D[c][1]}, {D[c][2]}, {D[c][3]}]\n' 
            rot = f'rotation = [ {R[c][0]}, {R[c][1]}, {R[c][2]}]\n'
            tran = f'translation = [ {T[c][0]}, {T[c][1]}, {T[c][2]}]\n'
            fish = f'fisheye = false\n\n'
            cal_f.write(cam + name + size + mat + dist + rot + tran + fish)
        meta = '[metadata]\nadjusted = false\nerror = 0.3\n'
        cal_f.write(meta)

C = ['M11139', 'M11140', 'M11141', 'M11458', 'M11459', 'M11461', 'M11462', 'M11463']

H, W = 1920, 1080
S = [[H,W], [H,W], [H,W], [H,W], [H,W], [H,W], [H,W], [H,W]]

K = np.load('/mnt/D494C4CF94C4B4F0/Trampoline_avril2025/Calib_trampo_avril2025/results_calib/calib_0425/Intrinsics_K_conf.npz')['arr_0']
D = np.load('/mnt/D494C4CF94C4B4F0/Trampoline_avril2025/Calib_trampo_avril2025/results_calib/calib_0425/Intrinsics_D_conf.npz')['arr_0']

K = K.squeeze()
D = D.squeeze()

world_T_cam = np.load('/mnt/D494C4CF94C4B4F0/Trampoline_avril2025/Calib_trampo_avril2025/results_calib/calib_0429/WorldTCam_opt.npz')['arr_0']
extrinsics = np.load('/mnt/D494C4CF94C4B4F0/Trampoline_avril2025/Calib_trampo_avril2025/results_calib/calib_0429/Extrinsics_optimized_20250429_12h00.npz')['arr_0']

R = []
for r in extrinsics[:, 0:3, 0:3]:
    rot_vec = cv2.Rodrigues(r)[0]
    R.append(rot_vec)
R = np.asarray(R).squeeze()
T = extrinsics[:, 0:3, 3] / 1000

toml_write('Calibration/Calib.toml', C, S, D, K, R, T)

In [ ]:
#Pose2Sim.poseEstimation()
#Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()

In [ ]:
# --- Configuration ---
cameras = ['Camera1_M11139', 'Camera2_M11140', 'Camera3_M11141', 'Camera4_M11458', 'Camera5_M11459', 'Camera6_M11461', 'Camera7_M11462', 'Camera8_M11463']
source_root = "pose_all_4dhumans"
pose2sim_pose_folder = "pose"
pose2sim_output_folder = "pose-3d"  # Where Pose2Sim outputs the .trc file
poseassociated_folder = "pose-associated"
dst_output_folder = "pose-3d-4DHumans"
os.makedirs(dst_output_folder, exist_ok=True)
sequence_names = set(list(f.split('-')[0] for f in os.listdir(source_root)))

def get_filename_parts(trc_path):
    """Split filename into parts excluding the pid (2nd field)"""
    base = os.path.basename(trc_path)
    parts = base.split('_')
    prefix, pid, suffix = parts
    return prefix, pid, suffix  # (prefix, rest)

wrong_seq = False
for seq in sorted(sequence_names):
    if seq not in [os.path.basename(f).split('.')[0] for f in os.listdir(dst_output_folder)]:
        # --- Step 1: Create/clear target folder ---
        if os.path.exists(pose2sim_pose_folder):
            shutil.rmtree(pose2sim_pose_folder)
        os.makedirs(pose2sim_pose_folder)

        if os.path.exists(poseassociated_folder):
            shutil.rmtree(poseassociated_folder)
        os.makedirs(poseassociated_folder)

        # --- Step 2: Search and move relevant files ---
        for cam_id in range(8):
            camera_folder_name = f"{seq}-{cameras[cam_id]}_json"
            src_path = os.path.join(source_root, camera_folder_name)
            dst_sequence_folder = os.path.join(pose2sim_pose_folder, camera_folder_name)

            if os.path.exists(src_path):
                shutil.copytree(src_path, dst_sequence_folder)
            else:
                wrong_seq = True
                break

        if wrong_seq:
            print(f"Skipping {seq} due to missing camera folder : {src_path}.")
            continue

        # --- Step 3: Run Pose2Sim triangulation pipeline ---
        Pose2Sim.personAssociation()
        Pose2Sim.triangulation()

        # --- Step 4: Rename and save the .trc file ---
        # Look for the latest .trc file in the output folder
        trc_files = sorted(glob.glob(os.path.join(pose2sim_output_folder, "*.trc")), key=os.path.getmtime)
        if not trc_files:
            raise FileNotFoundError("No .trc file found after triangulation.")

        latest_trc = trc_files[-1]

        new_trc_path = os.path.join(dst_output_folder, f"{seq}.trc")
        shutil.move(latest_trc, new_trc_path)
        
        # additional step for multi-person config
        """ # Step 2: Get latest .trc and its non-pid parts
        latest_trc = trc_files[-1]
        prefix, pid, suffix = get_filename_parts(latest_trc)

        # Step 3: Look backward for another .trc with same prefix and suffix (different pid)
        for trc in reversed(trc_files[:-1]):
            if '_P' in trc:
                _, pid2, s = get_filename_parts(trc)
                if s == suffix:
                    new_trc_path = os.path.join(dst_output_folder, f"{seq}-{pid2}.trc")
                    shutil.move(trc, new_trc_path) """

        print(f"TRC saved as: {new_trc_path}")

In [ ]:
import logging
# --- Configuration ---
cameras = ['Camera1_M11139', 'Camera2_M11140', 'Camera3_M11141',
           'Camera4_M11458', 'Camera5_M11459', 'Camera6_M11461',
           'Camera7_M11462', 'Camera8_M11463']

cam_suffix = [c.split('_')[-1] for c in cameras]

source_root = "pose_all_vit"
pose2sim_pose_folder = "pose"
pose2sim_output_folder = "pose-3d"
poseassociated_folder = "pose-associated"
dst_output_folder = "pose-3d-vit-multi"
os.makedirs(dst_output_folder, exist_ok=True)

sequence_names = set(f.split('-')[0] for f in os.listdir(source_root))
reprojection_errors = {}

def get_filename_parts(trc_path):
    base = os.path.basename(trc_path)
    parts = base.split('_')
    prefix, pid, suffix = parts
    return prefix, pid, suffix
    
def extract_reprojection_error(log_text):
    pattern = r"-->\s*Mean reprojection error for all points on .*? is ([\d.]+) px.*?([\d.]+) mm"
    match = re.search(pattern, log_text)
    if match:
        px = float(match.group(1))
        mm = float(match.group(2))
        return px, mm
    return None, None

def capture_pose2sim_logs():
    # Create StringIO to capture logs
    log_capture_string = io.StringIO()

    # Create a temporary logging handler
    ch = logging.StreamHandler(log_capture_string)
    ch.setLevel(logging.INFO)  # or DEBUG, depending on what's used
    formatter = logging.Formatter('%(message)s')
    ch.setFormatter(formatter)

    # Get the root logger used by Pose2Sim
    logger = logging.getLogger()
    logger.addHandler(ch)

    try:
        Pose2Sim.personAssociation()
        try:
            Pose2Sim.triangulation()
        except Exception as e:
            # Optionally log the exception manually
            logging.error(str(e))
            return 'error'
    finally:
        logger.removeHandler(ch)

    return log_capture_string.getvalue()

def update_calib_file(selected_camera_names):
    with open('Calibration/Calib_all.toml', 'r') as f:
        lines = f.readlines()

    writing = False
    filtered_lines = []

    for line in lines:
        if line.strip().startswith('[') and line.strip().endswith(']'):
            camera_id = line.strip()[1:-1]
            writing = camera_id in selected_camera_names

        if writing:
            filtered_lines.append(line)

    with open('Calibration/Calib.toml', 'w') as f:
        f.writelines(filtered_lines)

# Loop through camera combinations
for k in range(2, 8):  # from 2 to 7 cameras
    for cam_combination in itertools.combinations(enumerate(cameras), k):
        cam_indices, cam_names = zip(*cam_combination)
        cam_label = '+'.join(f"C{idx+1}" for idx in cam_indices)  # Label like C1+C3+C5

        for seq in sorted(sequence_names):
            out_folder = os.path.join(dst_output_folder, seq, cam_label)
            # if os.path.exists(os.path.join(out_folder, f"{seq}.trc")):
            #     continue  # Skip if already processed

            wrong_seq = False
            shutil.rmtree(pose2sim_pose_folder, ignore_errors=True)
            shutil.rmtree(poseassociated_folder, ignore_errors=True)
            os.makedirs(pose2sim_pose_folder, exist_ok=True)
            os.makedirs(poseassociated_folder, exist_ok=True)

            # Step 2: Copy only selected camera folders
            for idx in cam_indices:
                cam_name = cameras[idx]
                folder_name = f"{seq}-{cam_name}_json"
                src_path = os.path.join(source_root, folder_name)
                dst_path = os.path.join(pose2sim_pose_folder, folder_name)

                if os.path.exists(src_path):
                    shutil.copytree(src_path, dst_path)
                else:
                    print(f"Missing camera folder: {src_path}")
                    wrong_seq = True
                    break

            if wrong_seq:
                print(f"Skipping {seq} ({cam_label}) due to missing camera folder.")
                continue

            # Step 3: Adjust Calib.yaml file according to selected cameras
            selected_cam_names = [cam_suffix[i] for i in cam_indices]
            update_calib_file(selected_cam_names)

            # Step 3: Run Pose2Sim and extract error
            log_output = capture_pose2sim_logs()

            # Pose2Sim.personAssociation()
            # Pose2Sim.triangulation()

            if log_output == 'error':
                continue

            error = extract_reprojection_error(log_output)

            # Step 4: Handle result
            trc_files = sorted(glob.glob(os.path.join(pose2sim_output_folder, "*.trc")), key=os.path.getmtime)
            if not trc_files:
                print(f"No .trc found for {seq} ({cam_label})")
                continue

            latest_trc = trc_files[-1]
            os.makedirs(out_folder, exist_ok=True)
            dst_trc = os.path.join(out_folder, f"{seq}.trc")
            shutil.move(latest_trc, dst_trc)

            # Step 5: Save reprojection error
            reprojection_errors.setdefault(seq, {})[cam_label] = error
            print(cam_label, error)

# --- Optional: Save reprojection errors to file ---
with open(os.path.join(dst_output_folder, "vit_reprojection_errors_cam_combi.json"), "w") as f:
    json.dump(reprojection_errors, f, indent=2)


In [ ]:
frame_idx = 60
frame_name = f'{frame_idx:06d}'

with open(f'/home/lea/trampo/Pose2Sim/pose_all_vit/1_partie_0429_000-Camera3_M11141_json/frame_{frame_idx:05d}_{frame_name}.json', 'r') as f:
    vit_pts = json.load(f)['people'][0]['pose_keypoints_2d']

vit_pts = np.array(vit_pts).reshape((-1, 3))

with open(f'/home/lea/trampo/Pose2Sim/pose_all_4dhumans/1_partie_0429_000-Camera3_M11141_json/frame_{frame_name}.json', 'r') as f:
    hmr_pts = json.load(f)['people'][0]['pose_keypoints_2d']

hmr_pts = np.array(hmr_pts).reshape((-1, 3))

plt.figure(figsize=(15,7))
plt.scatter(vit_pts[:, 0], vit_pts[:, 1], s=3, label='vit')
plt.scatter(hmr_pts[:, 0], hmr_pts[:, 1], s=3, label='4DHumans')
plt.legend()
plt.xlim(0, 1920)
plt.ylim(0, 1080)
plt.show()

In [ ]:
line = "Mean reprojection error for all points on frames 60 to 309 is 10.0 px, which roughly corresponds to 60.1 mm."
match = re.search(r"([\d.]+)\s*px.*?([\d.]+)\s*mm", line)
print(match)

In [ ]:
%matplotlib widget
# --- Load and parse .trc file ---
def load_trc(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    # --- Metadata ---
    data_start_line = 5
    header_line = lines[3]
    header_parts = re.split(r'\s+', header_line.strip())
    
    marker_names = header_parts[2:]  # skip "Frame#" and "Time"
    n_markers = len(marker_names)
    
    # Reconstruct columns: Frame#, Time, then X/Y/Z for each marker
    cols = ['Frame#', 'Time']
    for name in marker_names:
        cols.append(f'{name}_X')
        cols.append(f'{name}_Y')
        cols.append(f'{name}_Z')

    # Read numerical data starting from line 6
    df = pd.read_csv(filename, sep=r'\s+|\t+', engine='python', skiprows=data_start_line, header=None)
    df.columns = cols

    return df, marker_names

def get_axis_limits(x_limits, y_limits, z_limits):
    # Find ranges and midpoints
    x_range = x_limits[1] - x_limits[0]
    y_range = y_limits[1] - y_limits[0]
    z_range = z_limits[1] - z_limits[0]
    max_range = max(x_range, y_range, z_range)

    x_mid = sum(x_limits) / 2
    y_mid = sum(y_limits) / 2
    z_mid = sum(z_limits) / 2

    return x_mid, y_mid, z_mid, max_range

# --- Visualization ---
def animate_markers(df, marker_names, stride=5):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    scatters = {}
    for name in marker_names:
        scatters[name], = ax.plot([], [], [], 'o', label=name)

    ax.view_init(elev=90, azim=-90)

    ax.set_xlim(-4, 4)
    ax.set_ylim(0, 8)
    ax.set_zlim(-2, 6)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    #ax.legend()

    def update(frame):
        for name in marker_names:
            x = df.loc[frame, f'{name}_X']
            y = df.loc[frame, f'{name}_Y']
            z = df.loc[frame, f'{name}_Z']
            scatters[name].set_data([x], [y])
            scatters[name].set_3d_properties([z])
        return scatters.values()

    ani = animation.FuncAnimation(fig, update, frames=range(0, len(df), stride), interval=50, blit=False)
    plt.show()

# --- Main ---
file_path = "/home/lea/trampo/Pose2Sim/pose-3d/Pose2Sim_0-1299.trc"  # Replace with the actual path
df, marker_names = load_trc(file_path)
animate_markers(df, marker_names)


In [ ]:
# --- Static 3D Plot for a Single Frame ---
def plot_frame_with_labels(df, marker_names, frame_idx=0):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')

    for name in marker_names:
        x = df.loc[frame_idx, f'{name}_X']
        y = df.loc[frame_idx, f'{name}_Y']
        z = df.loc[frame_idx, f'{name}_Z']
        ax.scatter(x, y, z, marker='o')
        ax.text(x, y, z, name, fontsize=8)

    ax.view_init(elev=90, azim=-90)

    """ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(4, 7)
    ax.set_zlim(-1, 2) """
    ax.axis('equal')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"Markers at Frame {frame_idx}")
    plt.show()

# Plot single frame with labels
plot_frame_with_labels(df, marker_names, frame_idx=1188)  # You can change the frame index

In [ ]:
def animate_skeleton(df, marker_names, connections, save_path='skeleton_animation.mp4'):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Get initial joint positions
    joints = {name: ax.plot([], [], [], 'o')[0] for name in marker_names}
    bones = [ax.plot([], [], [], 'black')[0] for _ in connections]

    def init():
        ax.view_init(elev=90, azim=-90)

        # Get min and max for each axis
        x_limits = df.filter(like='_X').values.min(), df.filter(like='_X').values.max()
        y_limits = df.filter(like='_Y').values.min(), df.filter(like='_Y').values.max()
        z_limits = df.filter(like='_Z').values.min(), df.filter(like='_Z').values.max()

        # Apply equal limits
        x_mid, y_mid, z_mid, max_range = get_axis_limits(x_limits, y_limits, z_limits)
        ax.set_xlim(x_mid - max_range/2, x_mid + max_range/2)
        ax.set_ylim(y_mid - max_range/2, y_mid + max_range/2)
        ax.set_zlim(z_mid - max_range/2, z_mid + max_range/2)
    
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        return list(joints.values()) + bones

    def update(frame):
        for name in marker_names:
            x = df.loc[frame, f'{name}_X']
            y = df.loc[frame, f'{name}_Y']
            z = df.loc[frame, f'{name}_Z']
            joints[name].set_data([x], [y])
            joints[name].set_3d_properties([z])

        for i, (a, b) in enumerate(connections):
            x = [df.loc[frame, f'{a}_X'], df.loc[frame, f'{b}_X']]
            y = [df.loc[frame, f'{a}_Y'], df.loc[frame, f'{b}_Y']]
            z = [df.loc[frame, f'{a}_Z'], df.loc[frame, f'{b}_Z']]
            bones[i].set_data(x, y)
            bones[i].set_3d_properties(z)

        return list(joints.values()) + bones

    anim = FuncAnimation(fig, update, frames=range(0, len(df), 4), init_func=init, blit=True)

    print("Saving animation...")
    writer = FFMpegWriter(fps=30, metadata=dict(artist='Pose2Sim'), bitrate=1800)
    anim.save(save_path, writer=writer)
    print(f"Saved to: {save_path}")

    plt.close(fig)

# --- Main ---
file_path = "pose-3d/Pose2Sim_0-1299.trc"
df, marker_names = load_trc(file_path)

print(marker_names)

# Define skeleton structure (example — adjust as needed)
connections = [
    ('RHip', 'RKnee'), ('RKnee', 'RAnkle'),
    ('LHip', 'LKnee'), ('LKnee', 'LAnkle'),
    ('RHip', 'LHip'), ('RHip', 'RShoulder'), ('LHip', 'LShoulder'),
    ('RShoulder', 'RElbow'), ('RElbow', 'RWrist'),
    ('LShoulder', 'LElbow'), ('LElbow', 'LWrist'),
    ('RShoulder', 'LShoulder'), ('Nose', 'RShoulder'), ('Nose', 'LShoulder')
]

animate_skeleton(df, marker_names, connections, save_path='pose_skeleton_4Dhumans.mp4')
plt.close()
